# Rabi Lenght

In [ ]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

In [ ]:
from qibolab.pulses import PulseSequence

sequence = PulseSequence()
qd_pulse = Device.create_qubit_drive_pulse(0, start=0, duration=1000) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=1000)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

In [ ]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np

def rabi_pulse_length(
    platform,
    qubit: int,
    pulse_duration_start,
    pulse_duration_end,
    pulse_duration_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Time": "ns"})

    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_drive_pulse(qubit, start=0, duration=4)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=4)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_duration_range = np.arange(
        pulse_duration_start, pulse_duration_end, pulse_duration_step
    )

    # platform.instruments["shfqc_qa"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.instruments["shfqc_qc"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - qd_pulse.frequency
    # )

    for _ in range(software_averages):
        for duration in qd_pulse_duration_range:
            qd_pulse.duration = duration
            ro_pulse.start = duration
    
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Time[ns]": duration,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data
            

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For Rabi oscillations
def time_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "popt4",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("Time", "ns"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("Time", "ns"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     timerange = np.linspace(
    #         min(data.get_values("Time", "ns")),
    #         max(data.get_values("Time", "ns")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=timerange,
    #             y=rabi(
    #                 timerange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.20,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} ns.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    # last part
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Time (ns)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Time (ns)",
        yaxis2_title="Phase (rad)",
    )
    return fig

In [ ]:
data = rabi_pulse_length(Device,
    qubit = 0,
    pulse_duration_start = 60,
    pulse_duration_end = 90,
    pulse_duration_step = 3,
    software_averages = 1,
)

time_msr_phase(data)

In [ ]:
Check Pi-pulse

In [ ]:
Device.reload_settings()
Device.native_gates["single_qubit"][0]["RX"]["duration"] = 80
Device.apply_settings()

In [ ]:
sequence = PulseSequence()
qd_pulse = Device.create_RX_pulse(0, start=0) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=qd_pulse.finish)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

------------------------------------

# Rabi Amplitude

Already at max Amp (Something wrong)

In [ ]:
def rabi_pulse_amp(
    platform,
    qubit: int,
    Amp_start,
    Amp_end,
    Amp_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Amp": "A"})

    sequence = PulseSequence()
    qd_pulse = platform.create_RX_pulse(qubit, start=0)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_amp_range = np.arange(
        Amp_start, Amp_end, Amp_step
    )

    for _ in range(software_averages):
        for Amp in qd_amp_range:

            qd_pulse.amplitude = Amp

            # platform.reload_settings()
            
            # platform.native_gates["single_qubit"][qubit]["RX"]["amplitude"] = Amp 

            # platform.apply_settings()

            # sequence = PulseSequence()
            # qd_pulse = platform.create_RX_pulse(qubit, start=0)
            # ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
            # sequence.add(qd_pulse)
            # sequence.add(ro_pulse)
            
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Amp[A]": Amp,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def amplitude_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = DataUnits()

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("amplitude", "dimensionless"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("amplitude", "dimensionless"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     amplituderange = np.linspace(
    #         min(data.get_values("amplitude", "dimensionless")),
    #         max(data.get_values("amplitude", "dimensionless")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=amplituderange,
    #             y=rabi(
    #                 amplituderange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.4f}",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Amplitude (dimensionless)",
        yaxis_title="MSR (uV)",
    )
    return fig

In [ ]:
data = rabi_pulse_amp(Device,
    qubit = 0,
    Amp_start = 0.8,
    Amp_end = 1,
    Amp_step = 0.3,
    software_averages = 1,
)

amplitude_msr_phase(data)

---------------------------------------------------

# Rabi Power

Already at max drive_range

In [ ]:
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np

def rabi_pulse_drive_range(
    platform,
    qubit: int,
    drive_range_start,
    drive_range_end,
    drive_range_step,
    software_averages,
):
    platform.reload_settings()
    platform.apply_settings()

    data = DataUnits(name=f"data_q{qubit}", quantities={"Drive_range": "dimensionless"})

    sequence = PulseSequence()
    qd_pulse = platform.create_qubit_RX_pulse(qubit, start=0)
    ro_pulse = platform.create_qubit_readout_pulse(qubit, start=qd_pulse.finish)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    qd_pulse_drive_range_range = np.arange(
        drive_range_start, drive_range_end,drive_range_step
    )

    # platform.instruments["shfqc_qa"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )
    # platform.instruments["shfqc_qc"]["settings"]["lo_frequency"] = (
    #     platform.characterization["single_qubit"][qubit]["qubit_freq"]
    #     - qd_pulse.frequency
    # )

    for _ in range(software_averages):
        for drive_range in qd_pulse_drive_range_range:
            
            platform.reload_settings()
            
            platform.instruments["shfqc_qc"]["settings"]["drive_range"] = drive_range
            
            platform.apply_settings()
    
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)
            msr, phase, i, q = platform.execute_pulse_sequence_NoSamples(sequence)

            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "Drive_range[dimensionless]": drive_range,
            }
            
            
            data.add(results)
            
        # Plot simulated output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return data

In [ ]:
def gain_msr_phase(data):

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "popt4",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data.get_values("gain", "dimensionless"),
            y=data.get_values("MSR", "uV"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data.get_values("gain", "dimensionless"),
            y=data.get_values("phase", "rad"),
            name="Rabi Oscillations",
        ),
        row=1,
        col=2,
    )

    # # add fitting trace
    # if len(data) > 0 and len(data_fit) > 0:
    #     gainrange = np.linspace(
    #         min(data.get_values("gain", "dimensionless")),
    #         max(data.get_values("gain", "dimensionless")),
    #         2 * len(data),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=gainrange,
    #             y=rabi(
    #                 gainrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #                 data_fit.get_values("popt4"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.20,
    #             showarrow=False,
    #             text=f"Estimated {params[1]} is {data_fit.df[params[1]][0]:.3f}",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"Estimated {params[0]} is {data_fit.df[params[0]][0]:.4f} uV",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )

    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Gain (dimensionless)",
        yaxis_title="MSR (uV)",
    )
    return fig
